In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
varinfo.clear()

In [4]:
module = fc.eightb.preselection.t8btag_minmass
dout = "yy_quadh"

In [5]:
altfile='yy_quadh_bkg10_soft025_md_{base}'
# signal = ObjIter([])
# signal = ObjIter([ Tree(f, report=False) for f in tqdm(module.signal_list) ])
signal = ObjIter([ Tree(f, report=False, altfile=altfile) for f in tqdm(module.full_signal_list) ])

bkg = ObjIter([])
# bkg = ObjIter([ Tree(module.Run2_UL18.QCD_B_List, altfile=altfile), Tree(module.Run2_UL18.TTJets, altfile=altfile)])

data = ObjIter([])
# data = ObjIter([ Tree(module.Run2_UL18.JetHT_Data_UL_List) ])

100%|██████████| 8/8 [00:38<00:00,  4.84s/it]


In [21]:
h4m = np.stack([ getattr(signal, f'{res}_m').apply(lambda m:m[:10000]).cat.to_numpy() for res in eightb.higgslist ], axis=1)
from scipy import optimize
def calc_dm(center, h4m=h4m):
        n = len(center)
        dm = np.sqrt( np.sum( ( (h4m[:,:n]-center)/center )**2, axis=-1 ) )
        return dm

def _find_best_(center):
    dm = calc_dm(center)
    mask = dm < 0.25
    return 1-np.mean(mask)
    # mu = ak.mean(dm[dm < 100])
    # return mu

r0 = (125,125,125,125)
center = optimize.fmin(_find_best_, r0,)
print(np.round(center, 2))

def set_higgs_rm(t, center=center):
    higgs_rm = {
            f'{res}_rm':t[f'{res}_m']/r
            for res,r in zip(eightb.higgslist,center)
        }
    t.extend(
        **higgs_rm,
        y1_h_rm = np.sqrt( (higgs_rm['H1Y1_rm']-1)**2 + (higgs_rm['H2Y1_rm']-1)**2),
        y2_h_rm = np.sqrt( (higgs_rm['H1Y2_rm']-1)**2 + (higgs_rm['H2Y2_rm']-1)**2)
    )
        